### **5.1** 试述将线性函数f(x)＝w^TX用作神经元激活函数的缺陷


需要注意的是，神经网络中必须要有非线性的激活函数，无论是在隐层，还是输出层，或者全部都是。如果用f(x)=ωTx做激活函数，无论多少层神经网络都退化成了线性回归。

### **5.2** 试述使用图5.2(b)激活函数的神经元与对率回归的联系

两者都是希望将连续值映射到{0,1}上，但由于阶跃函数不光滑，不连续的性质，所以才选择了sigmoid作为映射函数。不同之处在于激活函数不一定要使用sigmoid，只要是非线性的可导函数都可以使用

### **5.4** 试述式(5.6)中学习率的取值对神经网络训练的影响

如果学习率太低，每次下降的很慢，使得迭代次数非常多。 如果学习率太高，在后面迭代时会出现震荡现在，在最小值附近来回波动。

### **5.5** 试编程实现标准BP算法和积累BP,在西瓜数据集3.0上分别用这两个算法训练一个单隐层网络,并进行比较

编程实现一个标准bp算法和一个累积bp算法，用这两个算法训练一个带有单隐层的网络，并给出在西瓜数据集3.0上的效果
首先，单隐层网络，就是输入和输出层之间只有一层神经元的网络，见西瓜书P102图 5.7所示
标准bp算法，如书上P102-104所述，每次更新参数只使用一个样本，所以到达收敛所需要训练的次数会比较多。
累积bp算法，每次更新参数时，使用的是所有的训练样本，也就是一个epoch之后再进行参数更新。所以训练次数可能比较少

In [2]:
from bp5_5 import bpAccum,bpstand
'''
V:输入层权值　size: (X_shape[1],hideNum) 
V_b:输入层偏置　size:(1,hideNum)
W:隐藏层至输出层权值　size:(hideNum,Y_shape[1])
W_b:隐藏层至输出层偏置　size: (1,Y_shape[1])
rate=0.1
error=0.001
maxTrainNum=1000000
trainNum=0
loss=10
bpstand(hideNum):
①权值及偏执初始化
②循环(loss>error): 每次只处理一个样本　k
    1.正向传输：计算loss
    H=sigmod(输入*V-V_b)
    Y_=sigmode(H*W-W_b)
    loss=标准差
    2.标准BP:计算梯度并跟新参数
    g = Y_*(1-Y_)*(Y[k]-Y_)
    e = H*(1-H)*g.dot(W)
    W += rate*H.T.dot(g)
    W_b -= rate*g
    V += rate*X[k].reshape(1,X[k].size).T.dot(e)
    V_b -= rate*e
    
bpAccum(hideNum):
每次循环处理所有样本　其余与标准BP一致
'''
bpstand(5)
bpAccum(5)
#可以看出，在得到同样精度的情况下，累积bp算法的训练次数要比标准bp算法的训练次数少的多。

标准bp算法:
总训练次数： 85068
最终损失： [0.0009996]
累计bp算法:
总训练次数： 7684
最终损失： [0.00099964]


### **5.6**试设计一个BP改进算法,能通过动态调整学习率显著提升收敛速度，编程实现该算法，并选择两个UCI数据集与标准BP算法进行实验比较


#### BP算法可以完成非线性问题，因此具有很大的优势，但是也存在很大的问题，下面我们就来总结一下：
　          （1）易形成局部极小值而得不到全局最优
             （2）训练次数多，使的学习效率降低，收敛速度慢
             （3）隐节点的选取缺乏理论指导
             （4） 训练时学习新样本有遗忘旧样本的趋势
####  1.增加动量项
![公式](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5CDelta%20w%28t%29%3D%5Ceta%5Ccdot%20err%5Ccdot%20x%20&plus;%20%5Calpha%20%5Ccdot%20%5CDelta%20w%28t-1%29)
alpha称为动量系数，一般0< alpha < 1
#### 2.自适应调节学习率 ![字母](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5Ceta)
#### AdaGrad方法（Adaptive gradient 自适应梯度）
![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5CDelta%20w_t%20%3D%20-%5Cfrac%7B%5Ceta%20%7D%7B%5Csqrt%7B%5Csum_%7Bk%3D1%7D%5E%7Bt%7Dg_k%5E2%20&plus;%20%5Cvarepsilon%20%7D%7Dg_t)
![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20g_t)表示当前时刻梯度
###### 在同一时刻对于不同的参数，权值更新是不同的，随着更新距离之和的增加，学习速率也越来越小，这也符合在模型训练初期的，希望参数变化更多更快，而在模型训练后期，希望参数变化的更慢且值更小
#### Adadelta方法 
##### Adadelta是对Adagrad的扩展，最初方案依然是对学习率进行自适应约束，但是进行了计算上的简化。 Adagrad会累加之前所有的梯度平方，由上面公式可以看出，仍依赖于人工设置一个全局学习率，η设置过大的话，会使regularizer过于敏感，对梯度的调节太大，中后期，分母上梯度平方的累加将会越来越大，使gradient→0，使得训练提前结束，因此为了解决后期梯度很小的弊端，提出了Adadelta方法，而Adadelta只累加固定大小的项，并且也不直接存储这些项，仅仅是近似计算对应的平均值。即：
#### ![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5CDelta%20w_t%20%3D%20-%5Cfrac%7B%5Ceta%20%7D%7B%5Csqrt%7B%5Clambda%20*%5Csum_%7Bk%3D1%7D%5E%7Bt-1%7Dg_K%5E2&plus;%281-%5Clambda%20%29g_t%5E2%20&plus;%20%5Cvarepsilon%20%7D%7D*g_t)
##### 还是依赖全局的学习速率，后面人们就继续做了一定的处理，和近似牛顿迭代法后：![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20E%7Cg%5E2%7C_t%20%3D%20%5Clambda%20*%20E%7Cg%5E2%7C_%7Bt-1%7D%20&plus;%20%281-%5Clambda%20%29*g_t%5E2)
##### 这里是使用的均值进行处理的，进而可以得到：![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5CDelta%20w_t%20%3D%20-%5Cfrac%7B%5Csqrt%7B%5Csum_%7B%5Cvarphi%20%3D%201%7D%5E%7Bt-1%7D%5CDelta%20w_%5Cvarphi%20%7D%7D%7B%5Csqrt%7BE%7Cg%5E2%7C_t&plus;%5Cvarepsilon%20%7D%7D)
##### **RMSProp方法**（梯度的均方根 root mean squared (RMS)）
##### RMSprop可以算作Adadelta的一个特例：
当=0.5时![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20E%7Cg%5E2%7C_t%20%3D%20%5Clambda%20*%20E%7Cg%5E2%7C_%7Bt-1%7D%20&plus;%20%281-%5Clambda%20%29*g_t%5E2)就变为了求梯度平方和的平均数。 
如果再求根的话，就变成了RMS(均方根)： 
##### ![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20RMS%7Cg%7C_t%20%3D%20%5Csqrt%7BE%7Cg%5E2%7C_t&plus;%5Cvarepsilon%20%7D)                               
##### 此时，这个RMS就可以作为学习率η的一个约束：
##### ![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5Cfn_jvn%20%5CDelta%20w_t%20%3D%20-%5Cfrac%7B%5Ceta%20%7D%7BRMS%7Cg%7C_t%7D*g_t)
##### Adam
##### 本质上是带有动量项的RMSprop，它利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率。Adam的优点主要在于经过偏置校正后，每一次迭代学习率都有个确定范围，使得参数比较平稳。公式如下：
##### ![](https://img-blog.csdnimg.cn/20181112141756696.png)
##### ![](https://private.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20%5CDelta%20w_t%20%3D%20-%5Cfrac%7B%5Chat%7Bm_t%7D%7D%7B%5Csqrt%7B%5Chat%7Bn_t%7D%7D&plus;%5Cvarepsilon%20%7D*%5Ceta)
### **3.引入抖度因子**
#### 对激活函数进行改进
### **4.从梯度方面进行优化**
#### 牛顿法,海森矩阵法,共轭梯度法